In [15]:
import requests
import pandas as pd
import re

In [2]:
response = requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/')
app_ids_df = pd.DataFrame(response.json()['applist']['apps'])
app_ids = app_ids_df['appid'].to_numpy()
app_ids = app_ids[0:15000]

array([1941401, 1897482, 2112761, ..., 2519630, 2519650, 2519680])

In [3]:
def get_reviews(appid, params):
        url_start = 'https://store.steampowered.com/appreviews/'
        try:
            response = requests.get(url=url_start+str(appid), params=params, headers={'User-Agent': 'Mozilla/5.0'})
        except:
                return {'reviews' : []}
        return response.json() # return data extracted from the json response
reviews = []
cursor = '*'
params = { # https://partner.steamgames.com/doc/store/getreviews
    'json' : 1,
    'filter' : 'all', # sort by: recent, updated, all (helpfullness)
    'language' : 'english', # https://partner.steamgames.com/doc/store/localization
    'day_range' : 9223372036854775807, # shows reviews from all time
    'review_type' : 'all', # all, positive, negative
    'purchase_type' : 'all', # all, non_steam_purchase, steam
    'num_per_page' : 50,
    'cursor' : '*'.encode()
          }
for i, app_id in enumerate(app_ids):
    reviews += get_reviews(app_id, params)['reviews']
    if (i+1)%500 == 0:
        print(f'{i+1} of {len(app_ids)}: {len(reviews)} reviews')

500 of 15000: 975 reviews
1000 of 15000: 1795 reviews
1500 of 15000: 2745 reviews
2000 of 15000: 3732 reviews
2500 of 15000: 5115 reviews
3000 of 15000: 5952 reviews
3500 of 15000: 7007 reviews
4000 of 15000: 7911 reviews
4500 of 15000: 8792 reviews
5000 of 15000: 9753 reviews
5500 of 15000: 10532 reviews
6000 of 15000: 11359 reviews
6500 of 15000: 12245 reviews
7000 of 15000: 12920 reviews
7500 of 15000: 13382 reviews
8000 of 15000: 14342 reviews
8500 of 15000: 15014 reviews
9000 of 15000: 15915 reviews
9500 of 15000: 16084 reviews
10000 of 15000: 16359 reviews
10500 of 15000: 16639 reviews
11000 of 15000: 17206 reviews
11500 of 15000: 17905 reviews
12000 of 15000: 19378 reviews
12500 of 15000: 20665 reviews
13000 of 15000: 22633 reviews
13500 of 15000: 24786 reviews
14000 of 15000: 26862 reviews
14500 of 15000: 29417 reviews
15000 of 15000: 31760 reviews


In [27]:
reviews_df = pd.DataFrame(reviews)[['review', 'voted_up']]
reviews_df.dropna(inplace=True)
reviews_df["voted_up"] = reviews_df["voted_up"].astype(int) # converts True/False to 1/0
new_reviews = reviews_df.iloc[25000:] # slice dataset for new reviews to test later 

In [28]:
reviews_df.to_csv('~/Documents/steam-reviewer-bert/data/reviews.csv')
new_reviews.to_csv('~/Documents/steam-reviewer-bert/data/new_reviews.csv')